In [23]:
import numpy as np

# Question 1

A train is moving along a track, and its position at different time intervals has been
recorded. The data is stored in an ASCII text file with the first column representing time $t$ (in
seconds) and the second column representing the train's position $s(t)$ (in meters).

Your task is to write a Python program that performs both numerical differentiation and
numerical integration:

1. **Numerical Differentiation:** Compute the velocity of the train at each recorded time
   point using the second-order accurate finite difference method $O(h²)$ for all
   points, including the first and last points.
1. **Numerical Integration:** Compute the total distance travelled by the train by
   integrating the velocity over the given time period.

No library functions allowed for differentiation and integration


In [24]:
def load_data(file_name):
    data_points = np.genfromtxt(file_name, delimiter=None, skip_header=2)
    # make time and distance arrays to float
    time = data_points[:, 0].astype(float)
    distance = data_points[:, 1].astype(float)
    return np.array(time), np.array(distance)

In [25]:
def compute_vel(t, s):
    v = np.zeros(len(s))
    h = t[1] - t[0]
    for i in range(0, len(s)):
        if i <= len(s) - 3:
            # forward difference
            v[i] = (-3*s[i] + 4*s[i+1] - s[i+2]) / (2*h)
        else:
            # backward difference
            v[i] = (3*s[i] - 4*s[i-1] + s[i-2]) / (2*h)
    return v

### 1. Differentiation


In [26]:
t, s = load_data('Asssignment6_Q1.txt')

In [27]:
v = compute_vel(t, s)

In [28]:
print("Velocities:", v)

Velocities: [ 0.1  4.7  9.9 15.3 20.3 24.9]


### 2. Integration


In [29]:
def compute_ditance(t, v):
    s = 0
    for i in range(1, len(v)):
        # trapezoidal rule
        s += (v[i] + v[i-1])*(t[i]-t[i-1])/2
    return s

In [30]:
s = compute_ditance(t, v)
print("Distance:", s)

Distance: 62.69999999999999


# Question 2

You are given a partially implemented Python program that is supposed to compute
the second derivative of a function 𝑓(𝑥) using numerical differentiation. The function is
sampled at unevenly spaced intervals. Your task is to correct or complete the given code so
that it correctly computes the second derivative of 𝑓(𝑥) at each data point.
The function 𝑓(𝑥) is provided as an array of points in an ASCII text file.

You are expected to modify or add functionality to correctly compute the second derivative
using methods suitable for non-uniform intervals. You may use techniques such as
interpolation or higher-order finite differences.


In [31]:
# Function to compute the second derivative using finite differences for non-uniform intervals
def compute_second_derivative(x, f, is_equally_spaced=True):
    n = len(x)
    second_derivative = np.zeros(n)

    # Unequally spaced points using Lagrange interpolation method
    for i in range(1, n-1):
        x0, x1, x2 = x[i-1], x[i], x[i+1]
        f0, f1, f2 = f[i-1], f[i], f[i+1]

        second_derivative[i] = 2 * (
            f0 / ((x0 - x1) * (x0 - x2)) +
            f1 / ((x1 - x0) * (x1 - x2)) +
            f2 / ((x2 - x0) * (x2 - x1))
        )

    # Use forward difference for the first point
    h1 = x[1] - x[0]
    h2 = x[2] - x[1]
    second_derivative[0] = 2 * (
        (f[2] - f[1]) / (h2 * (h1 + h2)) -
        (f[1] - f[0]) / (h1 * (h1 + h2))
    )

    # Use backward difference for the last point
    h1 = x[-1] - x[-2]
    h2 = x[-2] - x[-3]
    second_derivative[-1] = 2 * (
        (f[-1] - f[-2]) / (h1 * (h1 + h2)) -
        (f[-2] - f[-3]) / (h2 * (h1 + h2))
    )

    return second_derivative

In [32]:
# Read data from file
def read_data(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()[2:]  # Skip the headers
        x = []
        f = []
        for line in lines:
            data = line.split()
            x.append(float(data[0]))
            f.append(float(data[1]))
    return np.array(x), np.array(f)

In [33]:
# Main function to test the second derivative computation
def main():
    file_name = 'Asssignment6_Q2.txt'
    x, f = read_data(file_name)

    # Compute second derivatives
    second_derivative = compute_second_derivative(x, f)

    # Print the results
    print("x         Second Derivative f''(x)")
    for i in range(len(x)):
        print(f"{x[i]:.2f}         {second_derivative[i]:.5f}")


if __name__ == '__main__':
    main()

x         Second Derivative f''(x)
0.00         2.85714
0.30         2.85714
0.70         -1.77778
1.20         0.33333
1.50         1.16667
2.00         1.16667


### Using scipy library to calculate the second derivative of a function


In [35]:
from scipy.interpolate import UnivariateSpline

file_name = 'Asssignment6_Q2.txt'
x, f = read_data(file_name)

# Create a UnivariateSpline object (smoothing spline)
spline = UnivariateSpline(x, f, k=2, s=0)

# Compute the second derivative
second_derivative = spline.derivative(n=2)

# Evaluate the second derivative at the original x points
second_grad = second_derivative(x)

print("Second Gradient:", second_grad)

Second Gradient: [ 3.85268788  3.85268788 -3.1161273   0.94493711  1.18960421  1.18960421]
